In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from unidecode import unidecode
import re

In [2]:


def fill_district(ward):
    parts = ward.split(', ')
    result = []
    current_district = None
    filled = []

    # Loop từ phải qua trái
    for part in reversed(parts):
        # Tìm district trong ngoặc
        m = re.search(r'\((.*?)\)', part)
        if m:
            current_district = m.group(1)
        # Nếu chưa có district thì bỏ qua
        if current_district:
            # Bỏ ngoặc cũ nếu có
            part = re.sub(r'\(.*?\)', '', part).strip()
            part = f"{part} ({current_district})"
        filled.append(part)

    # Đảo lại đúng thứ tự ban đầu
    filled = list(reversed(filled))
    return ', '.join(filled)

def split_district(truocsapnhap):
    unit = truocsapnhap.split(' (')
    district = np.nan
    if len(unit) > 1:
        district = unit[1][:-1]
    return district

def split_ward(truocsapnhap):
    unit = truocsapnhap.split(' (')
    ward = unit[0]
    return ward

def cap_first(text):
    if isinstance(text, str):
        split_text = text.split()
        split_text = [i.capitalize() if split_text.index(i)==0 else i for i in split_text]
        return ' '.join(split_text)
    else:
        return np.nan

def unidecode_pro(text):
    if isinstance(text, str):
        text = unidecode(text)
        text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)  # chỉ giữ a-z, A-Z, 0-9, thay phần còn lại bằng khoảng trắng
        text = re.sub(r'\s+', ' ', text)
        return text.strip().lower()
    else:
        return np.nan

In [3]:
df_ward = pd.read_csv('data/sapnhap.bando.com.vn_ward.csv')
df_ward['truocsapnhap'] = df_ward['truocsapnhap'].str.replace(r'\(phần còn lại.*\)', '', regex=True).str.strip()
df_ward['truocsapnhap'] = df_ward['truocsapnhap'].str.replace(r'\(bao gồm.*\)', '', regex=True).str.strip()
df_ward['truocsapnhap'] = df_ward['truocsapnhap'].str.replace(r'\(một phần.*\)', '', regex=True).str.strip()
df_ward['truocsapnhap'] = df_ward['truocsapnhap'].str.replace(r'\(do huyện.*\)', '', regex=True).str.strip()
df_ward['truocsapnhap'] = df_ward['truocsapnhap'].str.replace(' và ', ', ', regex=False).str.strip()
df_ward['truocsapnhap'] = df_ward['truocsapnhap'].str.replace('(thuộc huyện ', '(huyện ', regex=False).str.strip()
df_ward['truocsapnhap'] = df_ward['truocsapnhap'].str.replace('(huyện Châu Thành, tỉnh Bến Tre)', '(huyện Châu Thành)', regex=False).str.strip()
df_ward['truocsapnhap'] = df_ward['truocsapnhap'].str.replace('(huyện Châu Thành, tỉnh Trà Vinh)', '(huyện Châu Thành)', regex=False).str.strip()
df_ward['truocsapnhap'] = df_ward['truocsapnhap'].apply(fill_district)
df_ward['truocsapnhap'] = df_ward['truocsapnhap'].str.split(',\s*')
df_ward_unpivot = df_ward.explode('truocsapnhap').reset_index(drop=True)

In [4]:
# df_ward[df_ward['truocsapnhap'].fillna('').str.contains('huyện Châu Thàn')]
# df_ward_unpivot[df_ward_unpivot['truocsapnhap'].fillna('').str.contains('huyện Châu Thàn')]

In [5]:
df_ward_unpivot['ward'] = df_ward_unpivot['truocsapnhap'].apply(split_ward)
df_ward_unpivot['district'] = df_ward_unpivot['truocsapnhap'].apply(split_district)

In [6]:
count_old_ward = df_ward_unpivot.groupby(['tentinh', 'truocsapnhap'])['truocsapnhap'].count().reset_index(name='count_truocsapnhap')
duplicated_old_ward = count_old_ward[count_old_ward['count_truocsapnhap'] > 1]
duplicated_old_ward = duplicated_old_ward[duplicated_old_ward['truocsapnhap']!='Không sáp nhập']
duplicated_old_ward['is_dividend_ward'] = True
df_ward_unpivot_category = pd.merge(df_ward_unpivot, duplicated_old_ward, on=['tentinh', 'truocsapnhap'], how='left')
df_ward_unpivot_category['is_dividend_ward'].fillna(False, inplace=True)

In [7]:
# Check sai ten khong
df_ward_unpivot_category['district'].drop_duplicates()

0                       NaN
14        quận Hai Bà Trưng
62          huyện Phú Xuyên
67              huyện Ba Vì
91          huyện Thanh Trì
                ...        
10232      thị xã Vĩnh Châu
10253         huyện Long Mỹ
10258      thành phố Cà Mau
10263    thành phố Bạc Liêu
10299        thị xã Giá Rai
Name: district, Length: 308, dtype: object

In [8]:
df_ward_unpivot_category['district'] = df_ward_unpivot_category['district'].apply(cap_first)
df_ward_unpivot_category['ward'] = df_ward_unpivot_category['ward'].apply(cap_first)
df_ward_unpivot_category['tentinh'] = df_ward_unpivot_category['tentinh'].apply(cap_first)

rename_cols = {
    'is_dividend_ward': 'is_dividend_ward',
    'ward': 'ward',
    'district': 'district',

    'ma': 'new_wardCode',
    'loai': 'new_type',
    'tenhc': 'new_ward',
    'tentinh': 'new_province',

    'kinhdo': 'new_wardLon',
    'vido': 'new_wardLat'
}
df_ward_unpivot_category = df_ward_unpivot_category.rename(columns=rename_cols)[rename_cols.values()]

In [9]:
df_ward_unpivot_category['new_province'].replace('Tỉnh Ninh BÌnh', 'Tỉnh Ninh Bình', inplace=True)

In [10]:
df_ward_unpivot_category

,is_dividend_ward,ward,district,new_wardCode,new_type,new_ward,new_province,new_wardLon,new_wardLat
0,False,Xã Đông La,NaN,2,xã,An Khánh,Thủ đô Hà Nội,105.708,20.98760
1,True,Phường Dương Nội,NaN,2,xã,An Khánh,Thủ đô Hà Nội,105.708,20.98760
2,False,Phường Quán Thánh,NaN,3,phường,Ba Đình,Thủ đô Hà Nội,105.838,21.03860
3,False,Phường Trúc Bạch,NaN,3,phường,Ba Đình,Thủ đô Hà Nội,105.838,21.03860
4,True,Phường Cửa Nam,NaN,3,phường,Ba Đình,Thủ đô Hà Nội,105.838,21.03860
...,...,...,...,...,...,...,...,...,...
10428,False,Xã Vĩnh Phú Tây,NaN,3353,xã,Vĩnh Phước,Tỉnh Cà Mau,105.410,9.39891
10429,False,Xã Hưng Phú,NaN,3354,xã,Vĩnh Thanh,Tỉnh Cà Mau,105.519,9.35997
10430,False,Xã Vĩnh Thanh,NaN,3354,xã,Vĩnh Thanh,Tỉnh Cà Mau,105.519,9.35997
10431,False,Phường 5,Thành phố Bạc Liêu,3355,phường,Vĩnh Trạch,Tỉnh Cà Mau,105.774,9.29729


# Province

In [11]:
df_province = pd.read_csv('data/sapnhap.bando.com.vn_province.csv')
df_province['truocsapnhap'] = df_province['truocsapnhap'].str.replace(' và ', ', ')
df_province['truocsapnhap'] = df_province['truocsapnhap'].str.split(',\s*')
df_province_unpivot = df_province.explode('truocsapnhap').reset_index(drop=True)
df_province_unpivot['province'] = np.where(df_province_unpivot['truocsapnhap']=='không sáp nhập', df_province_unpivot['tentinh'], df_province_unpivot['truocsapnhap'])
df_province_unpivot['is_merged_province'] = np.where(df_province_unpivot['truocsapnhap']=='không sáp nhập', False, True)

In [12]:
rename_cols = {
    'mahc': 'new_provinceCode',
    'tentinh': 'new_province',
    'province': 'province',
    'is_merged_province': 'is_merged_province'
}
df_province_unpivot_mini = df_province_unpivot.rename(columns=rename_cols)[rename_cols.values()]
df_province_unpivot_mini['province'] = df_province_unpivot_mini['province'].apply(cap_first)
df_province_unpivot_mini['new_province'] = df_province_unpivot_mini['new_province'].apply(cap_first)

In [13]:
df_province_unpivot_mini

,new_provinceCode,new_province,province,is_merged_province
0,1,Thủ đô Hà Nội,Thủ đô Hà Nội,False
1,7,Tỉnh Cao Bằng,Tỉnh Cao Bằng,False
2,8,Tỉnh Tuyên Quang,Tỉnh Hà Giang,True
3,8,Tỉnh Tuyên Quang,Tỉnh Tuyên Quang,True
4,9,Tỉnh Lào Cai,Tỉnh Yên Bái,True
...,...,...,...,...
58,33,Thành phố Cần Thơ,Thành phố Cần Thơ,True
59,33,Thành phố Cần Thơ,Tỉnh Sóc Trăng,True
60,33,Thành phố Cần Thơ,Tỉnh Hậu Giang,True
61,34,Tỉnh Cà Mau,Tỉnh Bạc Liêu,True


# Danh mục

In [14]:
df_danhmuc = pd.read_csv('data/danhmuchanhchinh.gso.gov.vn_2025-07-18.csv')
rename_cols = {
    'Mã': 'wardCode',
    'Mã QH': 'districtCode',
    'Mã TP': 'provinceCode',
    'Cấp': 'wardType',
    'Tên': 'ward',
    'Quận Huyện': 'district',
    'Tỉnh / Thành Phố': 'province'
}
df_danhmuc = df_danhmuc.rename(columns=rename_cols)[rename_cols.values()]

In [15]:
df_danhmuc_district = pd.read_csv('data/danhmuchanhchinh.gso.gov.vn_district_2025-07-18.csv')
rename_cols = {
    'Mã': 'districtCode',
    'Mã TP': 'provinceCode',
    'Cấp': 'districtType',
    'Tên': 'district',
    'Tỉnh / Thành Phố': 'province'
}

df_danhmuc_district = df_danhmuc_district.rename(columns=rename_cols)[rename_cols.values()]

In [16]:
df_danhmuc_district.shape

(696, 5)

In [17]:
df_danhmuc[['province', 'district']].drop_duplicates().shape

(691, 2)

In [18]:
df_danhmuc_district[~df_danhmuc_district['district'].isin(df_danhmuc['district'])]

,districtCode,provinceCode,districtType,district,province
223,318,31,Huyện,Huyện Bạch Long Vĩ,Thành phố Hải Phòng
339,471,45,Huyện,Huyện Cồn Cỏ,Tỉnh Quảng Trị
356,498,48,Huyện,Huyện Hoàng Sa,Thành phố Đà Nẵng
386,536,51,Huyện,Huyện Lý Sơn,Tỉnh Quảng Ngãi
539,755,77,Huyện,Huyện Côn Đảo,Tỉnh Bà Rịa - Vũng Tàu


In [19]:
common = list(set(df_danhmuc.columns.tolist()) & set(df_danhmuc_district.columns.tolist()))
df_danhmuc = pd.merge(df_danhmuc_district, df_danhmuc, on=common, how='left')

df_danhmucdf_danhmuc# Merge

In [20]:
df_province_unpivot_mini[~df_province_unpivot_mini['province'].isin(df_danhmuc['province'])]

,new_provinceCode,new_province,province,is_merged_province
0,1,Thủ đô Hà Nội,Thủ đô Hà Nội,False
14,12,Tỉnh Phú Thọ,Tỉnh Hòa Bình,True
48,29,Thành phố Hồ Chí Minh,Tphcm,True


In [21]:
df_province_unpivot_mini.loc[df_province_unpivot_mini['province']=='Thủ đô Hà Nội', 'province'] = 'Thành phố Hà Nội'
df_province_unpivot_mini.loc[df_province_unpivot_mini['province']=='Tphcm', 'province'] = 'Thành phố Hồ Chí Minh'
df_province_unpivot_mini.loc[df_province_unpivot_mini['province']=='Tỉnh Hòa Bình', 'province'] = 'Tỉnh Hoà Bình'

In [22]:
df_danhmuc_merge = pd.merge(df_danhmuc, df_province_unpivot_mini, on='province', how='left')

In [23]:
df_danhmuc_merge

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,new_provinceCode,new_province,is_merged_province
0,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,1.0,Phường,Phường Phúc Xá,1,Thủ đô Hà Nội,False
1,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,4.0,Phường,Phường Trúc Bạch,1,Thủ đô Hà Nội,False
2,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,6.0,Phường,Phường Vĩnh Phúc,1,Thủ đô Hà Nội,False
3,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,7.0,Phường,Phường Cống Vị,1,Thủ đô Hà Nội,False
4,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,8.0,Phường,Phường Liễu Giai,1,Thủ đô Hà Nội,False
...,...,...,...,...,...,...,...,...,...,...,...
10035,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32239.0,Xã,Xã Viên An Đông,34,Tỉnh Cà Mau,True
10036,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32242.0,Xã,Xã Viên An,34,Tỉnh Cà Mau,True
10037,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32244.0,Thị trấn,Thị trấn Rạch Gốc,34,Tỉnh Cà Mau,True
10038,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32245.0,Xã,Xã Tân Ân,34,Tỉnh Cà Mau,True


## Merge ward

In [24]:
df_ward_unpivot_category['district'] = np.where(df_ward_unpivot_category['ward'].str.contains(r'^Huyện'), df_ward_unpivot_category['ward'], df_ward_unpivot_category['district'])
df_ward_unpivot_category['ward'] = np.where(df_ward_unpivot_category['ward'].str.contains(r'^Huyện'), np.nan, df_ward_unpivot_category['ward'])

In [25]:
df_ward_unpivot_category['unidecode_district'] = df_ward_unpivot_category['district'].apply(unidecode_pro)
df_ward_unpivot_category['unidecode_ward'] = df_ward_unpivot_category['ward'].apply(unidecode_pro)

df_danhmuc_merge['unidecode_district'] = df_danhmuc_merge['district'].apply(unidecode_pro)
df_danhmuc_merge['unidecode_ward'] = df_danhmuc_merge['ward'].apply(unidecode_pro)

In [26]:
# pd.merge(df_danhmuc_merge, df_ward_unpivot_category_has_district, on=['new_province', 'unidecode_district', 'unidecode_ward'], how='left')
# pd.merge(df_danhmuc_merge, df_ward_unpivot_category_nan_district, on=['new_province', 'unidecode_ward'], how='left')

In [27]:
df_ward_unpivot_category[(~df_ward_unpivot_category['unidecode_district'].isin(df_danhmuc_merge['unidecode_district'])) & (~df_ward_unpivot_category['unidecode_district'].isna()) & (df_ward_unpivot_category['unidecode_district']!='khong sap nhap')].sort_values(by=['unidecode_district'], ascending=False)

,is_dividend_ward,ward,district,new_wardCode,new_type,new_ward,new_province,new_wardLon,new_wardLat,unidecode_district,unidecode_ward


In [32]:
spell_map_danhmuc = {
    'Thuỵ': 'Thụy',
    'Quí': 'Quý',
    'Xà Phìn': 'Sà Phìn',
    'Sĩ Bình': 'Sỹ Bình',
    'Nàn Xín': 'Nàn Sín',
    'Hát Lìu': 'Hát Lừu',
    'Sủng Tráng': 'Sủng Cháng',
    'Zuôich': 'Zuôih',
    'Săm Khóe': 'Xăm Khòe',
    'Quy Đức': 'Qui Đức',
    'Sáng Nhè': 'Xá Nhè',
    'Tông Qua Lìn': 'Tung Qua Lìn',
    'Tà Bhinh': 'Tà Bhing',
    'Xá Tổng': 'Sá Tổng',
    'Bờ Y': 'Pờ Y',
    'Ô Long Vỹ': 'Ô Long Vĩ',
    'Ngọc Yêu': 'Ngọk Yêu',
    'Ngọc Lây': 'Ngọk Lây',
    'Sán Xả Hồ': 'Sán Sả Hồ',
    'Noong Lay': 'Nong Lay',
    'H Bông': 'HBông',
    "H' Neng": 'Hneng',
    "Ea R'Bin": 'Ea Rbin',
    'EaBia': "Ea Bia",
    'An Quy': 'An Qui',
    'Huổi Lếnh': "Huổi Lếch",
    'Ia Ma Rơn': "Ia Mrơn",
    "Hua Bun": "Hua Bum",
    "Hòa Tú II": 'Hòa Tú 2',
    "Ealy": "Ea Ly",
    'Cư K Róa': "Cư Króa",
    'Cư Ê Wi': "Cư Êwi",
    'P Rao': "Prao",
    "Thị trấn NT": "Thị trấn Nông trường",
    "Thị trấn N.T": "Thị trấn Nông trường",
    "Cư Jang": "Cư Yang",
    "Bha Lê": "Bha Lêê",
    "Đạm Bri": "Đamb'ri",
    "Măng Buk": "Măng Bút",
    "K' Dang": "KDang",
    "Ea H'đinh": "Ea H'đing",
    "Ma Ly Pho": "Ma Li Pho",
    "Mà Cooi": "Mà Cooih",
    "Lục Sỹ Thành": "Lục Sĩ Thành",
    "Lả Nhì Thàng": "Lản Nhì Thàng",
    "Laêê": "La Êê",
    "Pú Pẩu": "Pú Bẩu",
    "Nậm Rạng": "Nậm Dạng",
    "EaTrol": "Ea Trol",
    "Đắk DRông": "Đắk D'rông",
    "Dế Su Phình": "Dế Xu Phình",
    "Đắk N'Dung": "Đắk N'Drung",
    "Đăk Roong": "Đăk Rong",
    "Đa Krông": "Đakrông",
    "Eachà Rang": "Ea Chà Rang",
}

spell_map_sapnhap = {
    'Xã Thu Thủy': 'Phường Thu Thủy',
    'Xí m Vàng': 'Xím Vàng',
    'Quí': 'Quý',
    'Avương': 'A Vương',
    'Atiêng': 'A Tiêng',
    'Anông': 'A Nông',
    'Ia Dreh': 'Ia HDreh',
    'Gari': 'Ga Ri',
    "Cư Dliê Mnông": "Cư Dliê M'nông",
    "Chơ GLong": "Chơ Long",
    "Axan": "A Xan",
    "Adơk": "A Dơk",
    "^Thủ Thừa": "Thị trấn Thủ Thừa",
    "Mađaguôi": "Ma Đa Guôi",
    "^Rạch Ông": "Phường Rạch Ông",
    "Phường Tiên Sơn": "Xã Tiên Sơn",
    "Thảo Ngu yên": "Thảo Nguyên",
    "^Phổ Vinh": "Phường Phổ Vinh",
    "^Phổ Ninh": "Phường Phổ Ninh",
    "^Phổ Minh": "Phường Phổ Minh",
    "^Phổ Hòa": "Phường Phổ Hòa",
    "^Krông Na": "Xã Krông Na",
    "^Nguyễn Trãi": "Phường Nguyễn Trãi",
    "^Lê Lợi": "Phường Lê Lợi",
    "Xã Đắc Sơn": 'Phường Đắc Sơn',
    "^Cư Yang": "Xã Cư Yang",
    "Can Hồ": "Kan Hồ",
    "Bhalêê": "Bha Lêê",
    "Đăk Plô": "Đăk Blô",
    "ĐamBri": "Đamb'ri",
    "Ea H’đing": "Ea H'đing",
    "N’ Thôn Hạ": "N'Thol Hạ",
    "Đưng K’Nớ": "Đưng KNớ",
    "Đắk D’rông": "Đắk D'rông",
    "Cư K’nia": "Cư Knia",
    "^Phường Đúc": "Phường Phường Đúc",
    "Đắk N’Drung": "Đắk N'Drung",
    "Phường IV": "Phường 4"
}

In [29]:
def correct_spell_danhmuc(text):
    if isinstance(text, str):
        for wrong, right in spell_map_danhmuc.items():
            # Thêm \b để đảm bảo khớp nguyên từ (word boundary)
            pattern = r'\b' + wrong + r'\b'
            text = re.sub(pattern, right, text)
    return text

def correct_spell_sapnhap(text):
    if isinstance(text, str):
        for wrong, right in spell_map_sapnhap.items():
            # Thêm \b để đảm bảo khớp nguyên từ (word boundary)
            pattern = r'\b' + wrong + r'\b'
            text = re.sub(pattern, right, text)
    return text

In [ ]:
# Do tên sai chỉ với một tỉnh, ở tỉnh khác tên đó vẫn đúng
df_danhmuc_merge.loc[(df_danhmuc_merge['province']=='Tỉnh Điện Biên') & (df_danhmuc_merge['ward']=='Xã Xín Chải'), 'ward'] = 'Xã Sín Chải'
df_danhmuc_merge.loc[(df_danhmuc_merge['province']=='Tỉnh Nghệ An') & (df_danhmuc_merge['ward']=='Xã Hưng Yên'), 'ward'] = 'Xã Hưng Yên Nam'
df_danhmuc_merge.loc[(df_danhmuc_merge['new_province']=='Tỉnh Hưng Yên') & (df_danhmuc_merge['ward']=='Xã Kim Chung'), 'ward'] = 'Xã Kim Trung'
df_ward_unpivot_category.loc[(df_ward_unpivot_category['new_province']=='Tỉnh Đồng Nai') & (df_ward_unpivot_category['ward']=='Xã Xuân An'), 'ward'] = 'Phường Xuân An'
df_ward_unpivot_category.loc[(df_ward_unpivot_category['new_province']=='Tỉnh Đồng Nai') & (df_ward_unpivot_category['ward']=='Xã Xuân Bình'), 'ward'] = 'Phường Xuân Bình'


df_danhmuc_merge['ward'] = df_danhmuc_merge['ward'].apply(correct_spell_danhmuc)
df_danhmuc_merge['unidecode_ward'] = df_danhmuc_merge['ward'].apply(unidecode_pro)

df_ward_unpivot_category['ward'] = df_ward_unpivot_category['ward'].apply(correct_spell_sapnhap)
df_ward_unpivot_category['unidecode_ward'] = df_ward_unpivot_category['ward'].apply(unidecode_pro)

In [30]:
new_province = "Tỉnh Tuyên Quang"
keyword = r'tan p'
condition_1 = (df_danhmuc_merge['unidecode_ward'].fillna('').str.contains(keyword))
condition_2 = (df_danhmuc_merge['new_province']==new_province)
df_danhmuc_merge[condition_1 & condition_2]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,new_provinceCode,new_province,is_merged_province,unidecode_district,unidecode_ward


In [31]:
df_ward_unpivot_category[(~df_ward_unpivot_category['unidecode_ward'].isin(df_danhmuc_merge['unidecode_ward'])) & (~df_ward_unpivot_category['unidecode_ward'].isna()) & (df_ward_unpivot_category['unidecode_ward']!='khong sap nhap')].sort_values(by=['unidecode_ward'], ascending=False)

,is_dividend_ward,ward,district,new_wardCode,new_type,new_ward,new_province,new_wardLon,new_wardLat,unidecode_district,unidecode_ward
6322,False,Xã Zuôih,NaN,1915,xã,Nam Giang,Thành phố Đà Nẵng,107.503,15.6769,NaN,xa zuoih
4605,False,Xã Xí m Vàng,NaN,567,xã,Xím Vàng,Tỉnh Sơn La,104.320,21.3584,NaN,xa xi m vang
3711,False,Xã Xăm Khòe,NaN,791,xã,Bao La,Tỉnh Phú Thọ,104.977,20.6463,NaN,xa xam khoe
4273,False,Xã Xá Nhè,NaN,443,xã,Sáng Nhè,Tỉnh Điện Biên,103.462,21.8456,NaN,xa xa nhe
1983,False,Xã Vũ Quý,NaN,1249,xã,Vũ Quý,Tỉnh Hưng Yên,106.382,20.4086,NaN,xa vu quy
...,...,...,...,...,...,...,...,...,...,...,...
6579,False,Phổ Hòa,NaN,2004,phường,Đức Phổ,Tỉnh Quảng Ngãi,108.959,14.8008,NaN,pho hoa
6525,False,Nguyễn Trãi,NaN,1983,phường,Đăk Bla,Tỉnh Quảng Ngãi,107.987,14.3230,NaN,nguyen trai
6524,False,Lê Lợi,NaN,1983,phường,Đăk Bla,Tỉnh Quảng Ngãi,107.987,14.3230,NaN,le loi
7337,False,Krông Na,NaN,2268,xã,Buôn Đôn,Tỉnh Đắk Lắk,107.679,12.9312,NaN,krong na


In [35]:
df_ward_unpivot_category[df_ward_unpivot_category['ward'].fillna('').str.contains('Xã Tân Phước')]

,is_dividend_ward,ward,district,new_wardCode,new_type,new_ward,new_province,new_wardLon,new_wardLat,unidecode_district,unidecode_ward
2614,False,Xã Tân Phước,NaN,203,xã,Đồng Tâm,Tỉnh Tuyên Quang,104.989,22.50080,NaN,xa tan phuoc
7874,False,Xã Tân Phước,NaN,2460,phường,Phước Hội,Tỉnh Lâm Đồng,107.731,10.66770,NaN,xa tan phuoc
8241,False,Xã Tân Phước Tây,NaN,2586,xã,Vàm Cỏ,Tỉnh Tây Ninh,106.541,10.50720,NaN,xa tan phuoc tay
8333,False,Xã Tân Phước,NaN,2617,xã,Đồng Tâm,Tỉnh Đồng Nai,107.021,11.57090,NaN,xa tan phuoc
9394,False,Xã Tân Phước,Huyện Tân Hồng,2986,xã,An Phước,Tỉnh Đồng Tháp,105.525,10.81840,huyen tan hong,xa tan phuoc
9477,False,Xã Tân Phước,Huyện Lai Vung,3013,xã,Lai Vung,Tỉnh Đồng Tháp,105.565,10.29830,huyen lai vung,xa tan phuoc
9593,False,Xã Tân Phước,Huyện Gò Công Đông,3050,xã,Tân Đông,Tỉnh Đồng Tháp,106.718,10.40190,huyen go cong dong,xa tan phuoc
10161,False,Xã Tân Phước Hưng,NaN,3253,xã,Tân Phước Hưng,Thành phố Cần Thơ,105.795,9.74639,NaN,xa tan phuoc hung


In [ ]:
df_ward_unpivot_category = df_ward_unpivot_category[~df_ward_unpivot_category['ward'].isin(['Trường bắn TB1', 'Khu vực bãi bồi ven biển'])]

In [ ]:
df_ward_unpivot_category_has_district_has_ward = df_ward_unpivot_category[(~df_ward_unpivot_category['district'].isna()) & (~df_ward_unpivot_category['ward'].isna())]

df_ward_unpivot_category_has_district_nan_ward = df_ward_unpivot_category[(~df_ward_unpivot_category['district'].isna()) & (df_ward_unpivot_category['ward'].isna())]

df_ward_unpivot_category_nan_district_has_ward = df_ward_unpivot_category[(df_ward_unpivot_category['district'].isna()) & (~df_ward_unpivot_category['ward'].isna())]

## Check duplidate & Exist

1. Check bộ có cả district và ward trước
- Danh sách khớp hoàn toàn. -> File 1: Có district và ward chuẩn
- Danh sách không khớp -> Do fill district thừa -> gán nan cho district -> File 2: Chỉ có ward

2. Check bộ chỉ có district:

Bộ có district và ward

In [ ]:

print('Tổng đơn vị cũ:', df_danhmuc_merge.shape[0])
col_1 = ['new_province', 'unidecode_district', 'unidecode_ward']
test_1 = df_ward_unpivot_category_has_district_has_ward[col_1].drop_duplicates()
print('Test 1:', test_1.shape[0])
left = pd.merge(df_danhmuc_merge, test_1, on=col_1, how='left').shape[0]
inner = pd.merge(df_danhmuc_merge, test_1, on=col_1, how='inner').shape[0]
print('Left:', left)
print('Inner:', inner)

Không bị dup, nhưng bị thiếu. Khả năng do fill district nên không đúng.

In [ ]:
col_2 = ['new_province', 'unidecode_district']
test_2 = df_ward_unpivot_category_has_district_nan_ward[col_2].drop_duplicates()
print('Test 2:', test_2.shape[0])
left = pd.merge(df_danhmuc_merge, test_2, on=col_2, how='left').shape[0]
inner = pd.merge(df_danhmuc_merge, test_2, on=col_2, how='inner').shape[0]
print('Left:', left)
print('Inner:', inner)

Không bị dup nhưng bị dư -> Bình thường, vì huyện đó trong danhmuc có nhiều ward thôi. -> File 3: Chỉ có district chuẩn rồi

Test bộ chỉ có ward

In [ ]:
col_3 = ['new_province', 'unidecode_ward']
test_3 = df_ward_unpivot_category_nan_district_has_ward[col_3].drop_duplicates()
print('Test 3:', test_3.shape[0])
left = pd.merge(df_danhmuc_merge, test_3, on=col_3, how='left').shape[0]
inner = pd.merge(df_danhmuc_merge, test_3, on=col_3, how='inner')
print('Left:', left)
print('Inner:', inner.shape[0])

In [ ]:
inner[inner.duplicated(subset=['new_province', 'unidecode_ward'], keep=False)].sort_values(by=['new_province', 'unidecode_ward'])

In [ ]:
test_3['new_province_unicode_ward'] = test_3['new_province'] + '_' + test_3['unidecode_ward']
df_danhmuc_merge['new_province_unicode_ward'] = df_danhmuc_merge['new_province'] + '_' + df_danhmuc_merge['unidecode_ward']
test_3[~test_3['new_province_unicode_ward'].isin(df_danhmuc_merge['new_province_unicode_ward'])]

# Check exist

In [ ]:
df_ward_unpivot_category_has_district_has_ward['unidecode_district_ward'] = df_ward_unpivot_category_has_district_has_ward['unidecode_district'] + '_' + df_ward_unpivot_category_has_district_has_ward['unidecode_ward']

df_danhmuc_merge['unidecode_district_ward'] = df_danhmuc_merge['unidecode_district'] + '_' + df_danhmuc_merge['unidecode_ward']

In [ ]:
df_ward_unpivot_category_has_district_has_ward[~df_ward_unpivot_category_has_district_has_ward['unidecode_district_ward'].isin(df_danhmuc_merge['unidecode_district_ward'])]

In [ ]:
df_danhmuc_merge[(df_danhmuc_merge['unidecode_district'].str.contains('huyen gia lam'))]

In [ ]:
df_danhmuc_merge[(df_danhmuc_merge['unidecode_ward'].fillna('').str.contains('xa dai thinh'))]

In [ ]:
df_ward_unpivot_category_nan_district_has_ward_part2 = df_ward_unpivot_category_has_district_has_ward[~df_ward_unpivot_category_has_district_has_ward['unidecode_district_ward'].isin(df_danhmuc_merge['unidecode_district_ward'])]

In [ ]:
df_ward_unpivot_category_nan_district_has_ward_part2['district'] = np.nan
df_ward_unpivot_category_nan_district_has_ward_part2['unidecode_district'] = np.nan

In [ ]:
col_4 = ['new_province', 'unidecode_ward']
test_4 = df_ward_unpivot_category_nan_district_has_ward_part2[col_4].drop_duplicates()
pd.merge(df_danhmuc_merge, test_4, on=col_4, how='left').shape[0]